# Multiple Linear Regression
If our columns are numeric, it's straightforward to add them to a linear regression model.

In [1]:
import pandas as pd
from utils import evaluate_model
from sklearn import linear_model

def train_multi_linear_model(training_set):
    def encode_data(data):
        return(data[['BedroomAbvGr','GrLivArea']])
    predictor = linear_model.LinearRegression()
    predictor.fit(encode_data(training_set),training_set['SalePrice'])
    def multi_linear_model(input_data):
        return(predictor.predict(encode_data(input_data)))
    return(multi_linear_model)

In [2]:
training_set = pd.read_csv("housing_price_data/training_data.csv")
multi_linear_model = train_multi_linear_model(training_set)
evaluate_model(multi_linear_model)

The model is inaccurate by $40051.22 on average.


40051.22131205686

Exercise: Add `LotArea` to the multilinear model

## Working with categorical variables (Part 1)
In order to make use of categorical data, we first need to encode it as a number. As a simple example, we'll encode the variable `CentralAir` as 0 if there is no central air conditioning, and 1 if there is central air conditioning. We can do this by using a boolean comparison operation, and relying on the fact that `True == 1`

In [3]:
def train_central_air_model(training_set):
    def encode_data(data):
        features = data.copy()
        features['CentralAir'] = features['CentralAir'] == 'Y'
        features = features[['BedroomAbvGr','GrLivArea','CentralAir']]
        return(features)

    training_features = encode_data(training_set)
    predictor = linear_model.LinearRegression()
    predictor.fit(training_features, training_set['SalePrice'])
    
    def central_air_model(input_data):
        input_features = encode_data(input_data)
        return(predictor.predict(input_features))
    return(central_air_model)

In [4]:
central_air_model = train_central_air_model(training_set)
evaluate_model(central_air_model)

The model is inaccurate by $38058.32 on average.


38058.31999115855

Exercise: Add whether or not the house has a pool to this model. (Look at the `PoolArea` variable)

# Working with ordinal variables
If the categories of a variable follow a clear rank, then we can label them by this rank. An example of this is the basement quality column.

    BsmtCond: Evaluates the general condition of the basement

       Ex	Excellent
       Gd	Good
       TA	Typical - slight dampness allowed
       Fa	Fair - dampness or some cracking or settling
       Po	Poor - Severe cracking, settling, or wetness

We would encode this as Po:1, Fa:2, TA:3, Gd:4, and Ex:5.

For houses without a basement (i.e. `BsmtCond is NaN`), we use a default value of 0


In [34]:
def train_basement_cond_model(training_set):
    def encode_data(data):
        features = data.copy()
        bsmt_cond_map = {'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1}
        features['BsmtQuality'] = features['BsmtCond'].map(bsmt_cond_map).fillna(0) # Some houses have no basement
        features = features[['BedroomAbvGr','GrLivArea','BsmtQuality']]
        return(features)

    training_features = encode_data(training_set)
    predictor = linear_model.LinearRegression()
    predictor.fit(training_features, training_set['SalePrice'])
    
    def basement_cond_model(input_data):
        input_features = encode_data(input_data)
        return(predictor.predict(input_features))
    return(basement_cond_model)

basement_cond_model = train_basement_cond_model(training_set)
evaluate_model(basement_cond_model)

Exercise: Build a linear regression model that incorporates KitchenQuality (see `KitchenQual` in `data_description.txt`)

# Working with categorical variables (part 2)


If there is no order, we use a technique called 'one hot encoding'. This involves creating a new column for each category

For example, the `Electrical` variable in the housing prices dataset contains the following categories:

       SBrkr	Standard Circuit Breakers & Romex
       FuseA	Fuse Box over 60 AMP and all Romex wiring (Average)	
       FuseF	60 AMP Fuse Box and mostly Romex wiring (Fair)
       FuseP	60 AMP Fuse Box and mostly knob & tube wiring (poor)
       Mix	Mixed

We would encode the table

|Electrical|
|---------|
  |FuseA|
  |FuseF|
  |FuseP|
  |Mix|
  |SBrkr|
  
as

|FuseA |FuseF|FuseP|Mix|SBrkr|
|-----|-----|-----|-----|
| 1| 0  | 0   | 0  | 0  |
| 0|1  | 0   | 0  | 0  |
| 0|0  | 1  | 0| 0|
| 0|0  | 0 | 1 |0 |
| 0|0  | 0 | 0 | 1|

In [38]:
def encode_electrical(electrical):
    one_hot_encoding = pd.DataFrame()
    one_hot_encoding['FuseA'] = electrical == 'FuseA'
    one_hot_encoding['FuseF'] = electrical == 'FuseF'
    one_hot_encoding['FuseP'] = electrical == 'FuseP'
    one_hot_encoding['Mix']   = electrical == 'Mix'
    one_hot_encoding['SBrkr'] = electrical == 'SBrkr'
    return(one_hot_encoding)


In [39]:
encode_electrical(training_set['Electrical']).head(20)

,FuseA,FuseF,FuseP,Mix,SBrkr
0,False,False,False,False,True
1,False,False,False,False,True
2,False,False,False,False,True
3,False,False,False,False,True
4,False,False,False,False,True
5,False,False,False,False,True
6,False,False,False,False,True
7,False,False,False,False,True
8,False,True,False,False,False
9,False,False,False,False,True


In [49]:
def train_electrical_model(training_set):
    def encode_data(data):
        electrical = encode_electrical(data['Electrical'])
        data = pd.concat([electrical,data[["BedroomAbvGr","GrLivArea"]]],axis=1)
        return(data)
    training_features = encode_data(training_set)
    predictor = linear_model.LinearRegression()
    predictor.fit(training_features, training_set['SalePrice'])
    
    def electrical_model(input_data):
        input_features = encode_data(input_data)
        return(predictor.predict(input_features))
    return(electrical_model)

electrical_model = train_electrical_model(training_set)
evaluate_model(electrical_model)


The model is inaccurate by $38909.20 on average.


38909.20325625401

Exericse: Encode `Heating` using One Hot encoding

# Scaling and combining features

Many machine learning algorithms require all variables to be on the same scale, ideally between -1 and 1. Let's compare LotArea to SalePrice

In [50]:
training_set[['LotArea','SalePrice']].describe()

,LotArea,SalePrice
count,1168.000000,1168.000000
mean,10521.577055,180590.277397
std,10678.605035,78815.697902
min,1300.000000,34900.000000
25%,7555.000000,129900.000000
50%,9423.000000,162950.000000
75%,11608.500000,214000.000000
max,215245.000000,755000.000000


We can transform these variables to be on the same scale using a preprocessing trick called _min/max scaling_. 

`MinMaxScale(X) = (X - min(X))/(max(X) - min(X))` 

In [52]:
from sklearn.preprocessing import MinMaxScaler

In [54]:
scaler = MinMaxScaler()
scaled = pd.DataFrame(scaler.fit_transform(training_set[['LotArea','SalePrice']]),
             columns=['LotArea','SalePrice'])
scaled.head()

,LotArea,SalePrice
0,0.033420,0.241078
1,0.038795,0.203583
2,0.046507,0.261908
3,0.038561,0.145952
4,0.060576,0.298709


In [55]:
scaled.describe()

,LotArea,SalePrice
count,1168.000000,1168.000000
mean,0.043103,0.202320
std,0.049913,0.109451
min,0.000000,0.000000
25%,0.029236,0.131926
50%,0.037968,0.177823
75%,0.048183,0.248715
max,1.000000,1.000000


We can put these back to their original scale using the `inverse_transform` method.

In [56]:
pd.DataFrame(scaler.inverse_transform(scaled),
             columns=["LotArea","SalePrice"]).head()

,LotArea,SalePrice
0,8450.0,208500.0
1,9600.0,181500.0
2,11250.0,223500.0
3,9550.0,140000.0
4,14260.0,250000.0


In [57]:
training_set[["LotArea","SalePrice"]].head()

,LotArea,SalePrice
0,8450,208500
1,9600,181500
2,11250,223500
3,9550,140000
4,14260,250000


We can also add new features to our model by combining two or more existing features. For example, let's
multiply bedrooms by bathrooms. 
We'll also scale our input and output.

In [60]:
def train_bedroom_bathroom_model(training_set):
    def encode_data(data,scaler = None):
        data = data.copy()
        data = data[['FullBath','BedroomAbvGr']]
        data['BedBath'] = data['FullBath'] * data['BedroomAbvGr']
        
        if(not scaler):
            scaler = MinMaxScaler()
            scaler.fit(data)
        pd.DataFrame(scaler.transform(data), columns = ['FullBath','BedroomAbvGr','BedBath'])
        return(data,scaler)
    
    training_features, scaler = encode_data(training_set)
    predictor = linear_model.LinearRegression()
    predictor.fit(training_features, training_set['SalePrice'])
    
    def bedroom_bathroom_model(input_data):
        input_features,_ = encode_data(input_data,scaler)
        predictions = predictor.predict(input_features)
        return(predictions)
    return(bedroom_bathroom_model)

In [61]:
bedroom_bathroom_model = train_bedroom_bathroom_model(training_set)
evaluate_model(bedroom_bathroom_model)

The model is inaccurate by $47140.90 on average.


47140.89624339486

Competition Exercise: Build a linear model to achieve as low score as possible. Winner will receive an Eliiza water bottle.